## LOAD DATASET (PREPROCESSED DATA)

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import  image_dataset_from_directory
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # To show all logs, including debug messages


In [2]:
# import torch
# print("CUDA available:", torch.cuda.is_available())
# print("GPU name:", torch.cuda.get_device_name(0))

In [3]:
PROCESSED_DATASET_PATH = r"Dataset/ProcessedData"

# GETTING THE DATASET FROM PREPROCESSED DATASET

In [4]:
dataset = image_dataset_from_directory(
        PROCESSED_DATASET_PATH,
        image_size = (224,224),
        batch_size = 32,        # it will take 32 images at ocne for processing
        labels = "inferred",     #name of the  class will be taken from the folder name
        label_mode = 'int',  #telling the model that output label will be integer type
        shuffle = True,
        seed = 42
)
            


Found 9778 files belonging to 99 classes.


In [5]:
train_size = int(0.8 * len(dataset))
val_size  = int(0.1 * len(dataset))
test_size = len(dataset)  - (train_size + val_size)


## ASSIGNING THE DATASETS TO THE SPECIFIC VARIABLES 


splitting the dataset to train dataset test dataset and validation dataset

In [6]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.take(test_size)
val_dataset = dataset.take(val_size)

In [7]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [9]:

# Dynamically determine the number of classes
INPUT_SHAPE = (224, 224, 3)  # Input shape of the images
EPOCHS = 10
LEARNING_RATE = 0.1


In [10]:

# Step 1: Define the Model
def create_model():
    model = models.Sequential([
        # Base Convolutional Layers
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
   
        

        # Flatten and Fully Connected Layers
        layers.Flatten(),
        # layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(101,activation='softmax')  # Final layer for classification
    ])
    return model


In [11]:

# Step 2: Compile the Model
model = create_model()

model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-5),
    loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
    metrics=['accuracy']
    
)


c:\Users\PUGAZH\Desktop\Projects\DeepLearning\FaceAgeDetection\dlvenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:

# Step 3: Set Up Callbacks
callbacks = [
    ModelCheckpoint(
        filepath='model_best.h5',  # Save the best model
        save_best_only=True,
        monitor='val_accuracy',  # Monitor validation accuracy
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        patience=5,  # Stop training if no improvement for 5 epochs
        monitor='val_loss',
        mode='min',
        verbose=1,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,  # Reduce learning rate by a factor of 0.2
        patience=3,  # If no improvement for 3 epochs
        min_lr=1e-6,
        verbose=1
    )
]



In [13]:
# Step 4: Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    
    callbacks=callbacks,
    verbose=1
)



Epoch 1/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.0705 - loss: 82.5089
Epoch 1: val_accuracy improved from -inf to 0.13229, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 77s 309ms/step - accuracy: 0.0706 - loss: 82.2677 - val_accuracy: 0.1323 - val_loss: 6.6830 - learning_rate: 0.0010
Epoch 2/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - accuracy: 0.1214 - loss: 6.4864
Epoch 2: val_accuracy improved from 0.13229 to 0.13542, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - accuracy: 0.1214 - loss: 6.4854 - val_accuracy: 0.1354 - val_loss: 5.5469 - learning_rate: 0.0010
Epoch 3/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.1296 - loss: 5.5177
Epoch 3: val_accuracy improved from 0.13542 to 0.13646, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 67s 275ms/step - accuracy: 0.1296 - loss: 5.5172 - val_accuracy: 0.1365 - val_loss: 4.9604 - learning_rate: 0.0010
Epoch 4/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.1335 - loss: 5.0390
Epoch 4: val_accuracy improved from 0.13646 to 0.14583, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 67s 274ms/step - accuracy: 0.1335 - loss: 5.0387 - val_accuracy: 0.1458 - val_loss: 4.6567 - learning_rate: 0.0010
Epoch 5/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - accuracy: 0.1454 - loss: 4.6941
Epoch 5: val_accuracy improved from 0.14583 to 0.16042, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 69s 284ms/step - accuracy: 0.1454 - loss: 4.6939 - val_accuracy: 0.1604 - val_loss: 4.3231 - learning_rate: 0.0010
Epoch 6/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.1491 - loss: 4.4307
Epoch 6: val_accuracy did not improve from 0.16042
244/244 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - accuracy: 0.1490 - loss: 4.4306 - val_accuracy: 0.1479 - val_loss: 4.2619 - learning_rate: 0.0010
Epoch 7/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.1465 - loss: 4.2972
Epoch 7: val_accuracy improved from 0.16042 to 0.17708, saving model to model_best.h5


244/244 ━━━━━━━━━━━━━━━━━━━━ 114s 467ms/step - accuracy: 0.1465 - loss: 4.2972 - val_accuracy: 0.1771 - val_loss: 4.1143 - learning_rate: 0.0010
Epoch 8/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.1580 - loss: 4.1936
Epoch 8: val_accuracy did not improve from 0.17708
244/244 ━━━━━━━━━━━━━━━━━━━━ 66s 271ms/step - accuracy: 0.1580 - loss: 4.1937 - val_accuracy: 0.1646 - val_loss: 4.0118 - learning_rate: 0.0010
Epoch 9/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.1504 - loss: 4.1156
Epoch 9: val_accuracy did not improve from 0.17708
244/244 ━━━━━━━━━━━━━━━━━━━━ 66s 270ms/step - accuracy: 0.1504 - loss: 4.1155 - val_accuracy: 0.1615 - val_loss: 3.9227 - learning_rate: 0.0010
Epoch 10/10
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.1517 - loss: 4.0695
Epoch 10: val_accuracy did not improve from 0.17708
244/244 ━━━━━━━━━━━━━━━━━━━━ 67s 274ms/step - accuracy: 0.1517 - loss: 4.0695 - val_accuracy: 0.1750 - val_loss: 3.8850 - learning_rate: 0.0010
Rest

In [14]:
# Step 5: Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.1698 - loss: 3.8907
Test Accuracy: 0.19


In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

def predict_age_from_image(image_path, model_path='model_best.h5', image_size=(224, 224)):
    """
    Given an image file path, this function will:
    1. Load the pre-trained model from a specified path.
    2. Preprocess the image (resize and normalize).
    3. Make a prediction on the age.
    4. Return the predicted age.
    
    Parameters:
    - image_path: Path to the image file for which the age is to be predicted.
    - model_path: Path to the saved trained model (.h5 format).
    - image_size: Target size for resizing the image to match the model input (default: (224, 224)).
    
    Returns:
    - predicted_age: The predicted age (a discrete value).
    """
    
    try:
        # ------------------- Step 1: Load the Trained Model ------------------- #
        model = load_model(model_path)  # Load the model from the specified path
        print(f"Model loaded successfully from {model_path}")
        
    except Exception as e:
        print(f"Error loading the model: {e}")
        return None
    
    try:
        # ------------------- Step 2: Preprocess the Input Image ------------------- #
        # Load the image
        image = load_img(image_path, target_size=image_size)  # Resize the image
        image = img_to_array(image)  # Convert image to a NumPy array
        
        # Normalize the image (scale pixel values to range [0, 1])
        image = tf.cast(image, tf.float32) / 255.0
        
        # Add a batch dimension (model expects a batch of images, not just one)
        image = np.expand_dims(image, axis=0)  # Shape: (1, 224, 224, 3)
        print(f"Image preprocessed successfully: {image.shape}")
        
    except Exception as e:
        print(f"Error preprocessing the image {image_path}: {e}")
        return None
    
    try:
        # ------------------- Step 3: Make the Prediction ------------------- #
        # Predict the output using the model (returns a probability distribution over ages)
        prediction = model.predict(image)
        
        # Get the index of the highest probability (i.e., the predicted age)
        predicted_age = np.argmax(prediction)  # Index with the highest probability
        print(f"Predicted age: {predicted_age}")
        
        return predicted_age  # Return the predicted age
        
    except Exception as e:
        print(f"Error making prediction for {image_path}: {e}")
        return None

# Example usage:
image_path = 'path_to_image.jpg'  # Replace with the actual image path
predicted_age = predict_age_from_image(r"C:\Users\PUGAZH\Desktop\Projects\DeepLearning\FaceAgeDetection\src\Dataset\face_age\028\41.png")

if predicted_age is not None:
    print(f"The predicted age for the given image is: {predicted_age}")


Model loaded successfully from model_best.h5
Image preprocessed successfully: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted age: 25
The predicted age for the given image is: 25
